#  Train and convert a GPT-2 Model into a TFjs Model w/ GPU For Free

Based on the [Colabatory](https://colab.research.google.com/drive/1VLG8e7YSEwypxU-noRNhsv5dW4NfTGce) made by [Max Woolf](http://minimaxir.com)

Updated by [Fuad Balashov](https://Fbalashov.com)

*Last updated: February 22th, 2021*

This Colabatory builds on Max's work to retrain an advanced text generating neural network on any text dataset using `gpt-2-simple`. He built this in a way that is free on a GPU. I adapted his work to transform the model that you generate into a tensorflow js model. While I was able to load and run the model in a browser, I didn't know how to input my values or interpret the output values. You can find the details on my work in [this blog post](https://blog.fbalashov.com/2021/02/a-gpt-2-model-on-browser-close-but-no.html). Hopefully someone can continue the effort! My additions begin in the section titled "".

Note that I also removed a sections on batch generation below.

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read my [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [ ]:
!nvidia-smi

Sun Feb 21 14:11:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first.

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80/P4 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [ ]:
gpt2.download_gpt2(model_name="124M")

## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [ ]:
gpt2.mount_gdrive()

Mounted at /content/drive


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [ ]:
file_name = "input.txt"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [ ]:
gpt2.copy_file_from_gdrive(file_name)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies.

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              learning_rate=1e-5,
              steps=1000,
              restore_from='fresh',
              run_name='run',
              print_every=10,
              sample_every=200,
              save_every=500
              )

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run2')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run')

Loading checkpoint checkpoint/run2med/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run2med/model-1000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [ ]:
gpt2.generate(sess,
    length=100,
    temperature=0.7,
    prefix="you run",
    nsamples=5,
    truncate='<|endoftext|>',
    batch_size=5,
    run_name='run')

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [ ]:
gpt2.generate(sess,
              length=100,
              temperature=0.7,
              prefix="Stepping back",
              nsamples=5,
              batch_size=5
              )

# Converting to a TensorFlow JS Model

Tensorflow js enables you to run a model directly in the browser. Most models you create with Tensorflow js will be pretty small, the one I generated was about 500MB which is pretty large. This may be possible to shrink with some understanding of the transformer.

You could also use a tensorflow js model in nodejs. This can enable you to run the model in react native or other environments in which you can't run a python model.

The transformer I used was made by [huggingface](https://huggingface.co/transformers/). It is built to transform PyTorch models to Tensorflow models and back. It can also convert the checkpoint we created into a pytorch model.

However, you must be using a newer version of tensorflow ([2.1+](https://github.com/tensorflow/tfjs/issues/3582)) than the rest of this Colabatory above. As a result, first thing is first, we need to restart the runtime, `Runtime > Restart runtime` in the menu. After you do that, come back here and reinstall gpt2 and We then need to install the transformers and a new version of tensorflow js.

In [ ]:
# Install the hugging face transformers
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install tensorflow tensorflowjs

#Re install gpt2 and reimport missing imports
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Now that we have these all installed were ready to turn the model we have trained into a pytorch model. I based these steps on [this answer from apeguero1](https://github.com/minimaxir/gpt-2-simple/issues/178#issuecomment-599067273).

In [ ]:
from transformers import GPT2Config, GPT2Model
index_path = '/content/checkpoint/run2/model-1000.index'
config = GPT2Config.from_pretrained('gpt2')
model = GPT2Model.from_pretrained(index_path,from_tf=True,config=config)
model.save_pretrained(save_directory="./hfModelrun")

Then we convert the pytorch model into a tensorflow (non-js) model. this and the next step are based on [this Stack Overflow post](https://stackoverflow.com/questions/62677651/openai-gpt-2-model-use-with-tensorflow-js) with the change being to use the model we created in the previous step instead of the original GPT2 model.

In [ ]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
import tensorflowjs
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# add the EOS token as PAD token to avoid warnings
model = TFGPT2LMHeadModel.from_pretrained(pretrained_model_name_or_path="/content/hfModelrun", from_pt=True, pad_token_id=tokenizer.eos_token_id)
model.save("./run_tensorflowOutput")

And now, it is time to create our Tensorflow js model! We are going to use the same steps from [the SO post in the previous step](https://stackoverflow.com/questions/62677651/openai-gpt-2-model-use-with-tensorflow-js) to generate the model. Here is [a page describing the files](https://www.tensorflow.org/js/tutorials/conversion/import_saved_model) you will find in the `web_model` folder.
Next, we will export this model from drive and see how to use this in the browser.

In [ ]:
!tensorflowjs_converter \
    --input_format=tf_saved_model \
    --output_node_names='gpt2' \
    --saved_model_tags=serve \
    /content/run_tensorflowOutput \
    /content/run_tensorflowOutput_web_model

We now need to copy our file to drive, download it to our machine and add it to our web application. Because the model was so large (500MB with either the medium or small gpt2 datasets) I ended up testing the model locally instead of adding it to my application. This is easiest to do on a recent version of Firefox. The large size should also be a consideration when copying files to drive.

First, copy the file to your google drive, I just drag and droped the folder over to "drive > My Drive" in the left panel.

Then go to your google drive and download the model (it could take a while if you have a slow connection).

At that point you can add it to your website and use it. The code below is based on [an example provided by google](https://www.tensorflow.org/js/tutorials/conversion/import_saved_model#step_2_loading_and_running_in_the_browser) as well as a rudimentary understanding of Tensorflow model.

```html
<html>
<head>
<script src="https://unpkg.com/@tensorflow/tfjs"></script>
</head>
<body>
    <script>
    async function run() {
        // Load up our model
        // https://js.tensorflow.org/api/1.0.0/#loadGraphModel
        const model = await tf.loadGraphModel('run_tensorflowOutput_web_model/model.json');
        // This was the shape and dtype I needed to provide... I have no idea what it means.
        input = tf.tensor([70,1,1,1,1], [1,5], 'int32');
        // I also tried execute and got the same result.
        const predictions = model.predict(input, [1,1]);
        // Similarly I have no idea what the massive array of floats returned mean
        console.log(predictions[0].dataSync());
    }
    run();
    </script>
</body>
</html>
```
If you want to run this locally I recommend using firefox as it is the easiest browser to enable filesystem loading. **This is not a setting you want to leave on!**. In short you will have to navigate to `about:config` in the address bar and temporarily set the `privacy.file_unique_origin` flag to false.

Then open the console, reload the page and wait for a FloatArray to print. That is as far as I got, hopefully someone can take this another step forward :)

If you want more detailed steps with pictures you can check out [the blog post](https://blog.fbalashov.com/2021/02/a-gpt-2-model-on-browser-close-but-no.html) that accompanies this colabatory.

### Important Note
If you do decide to deploy this on a website **prompt your user before downloading the model**. Many may not want to load a 500MB model in case their connection is slow or they are on a metered connection. Unfortunatley [the api to check if a user is on a metered connection](https://developer.mozilla.org/en-US/docs/Web/API/Navigator/connection) isn't supported on IOS yet so you need to fall back to a user prompt.


### Other Resources
* [The page from google on Tensors](https://js.tensorflow.org/api/latest/).
* [HuggingFace's page on GPT2 models](https://huggingface.co/transformers/model_doc/gpt2.html)
* [Someone built logic to use a Bert model in TFjs which takes string args](https://github.com/tensorflow/tfjs-models/blob/master/qna/src/question_and_answer.ts). The process they use to tokenize args, and interpret results could be applicable!
* [An example covering tokenization for a Bert model from HuggingFace](https://blog.tensorflow.org/2020/05/how-hugging-face-achieved-2x-performance-boost-question-answering.html).

---

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [ ]:
model_name = "774M"

gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 354Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 131Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 279Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 3.10Git [00:23, 131Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 380Mit/s]                                                
Fetching model.ckpt.meta: 2.10Mit [00:00, 226Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 199Mit/s]                                                       


In [ ]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

W0828 18:37:58.571830 139905369159552 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Loading pretrained model models/774M/model.ckpt


In [ ]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="The secret of life is",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2021 Fuad Balashov

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.